In [29]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.layers import Input, Dense, Dropout, Conv3D, ReLU
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from keras.models import Sequential, Model

In [26]:
resnet = ResNet50(weights='imagenet', pooling='avg', include_top=False)

# ResNet for extracting features, which is also the encoder 
img_path = 'skiing.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = resnet.predict(x)
preds.shape
# preds is the feature encoded by ResNet

(1, 2048)

In [39]:
def build_regressor():
  # the input here is the feature encoded by ResNet
  input_layer = Input(shape=([2048]),name = 'ResNet_feature_input')
  dense1 = Dense(1024, activation='relu', name='fully_connceted_1')(input_layer)
  dropout1 = Dropout(0.2)(dense1)
  dense2 = Dense(1024, activation='relu', name='fully_connected_2')(dropout1)
  encoder_end = Dense(85, activation='relu', name='fully_connected_3')(dense2)

  model = Model(inputs=[input_layer], outputs=[encoder_end])
  model.summary()
  return model

In [40]:
preds.shape

(1, 2048)

In [44]:
def build_discriminator():

  input_layer = Input(shape=([2048]), name='dis_input')
  dense1 = Dense(10, activation='relu', name='fully_connceted_1_dis')(input_layer)
  dense2 = Dense(5, activation='relu', name='fully_connceted_2_dis')(dense1)
  validity = Dense(1, activation='sigmoid', name='final_dis')(dense2)
  model = Model(inputs=[input_layer], outputs=validity)
  model.summary()

  return model

In [45]:
build_regressor()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
ResNet_feature_input (InputL (None, 2048)              0         
_________________________________________________________________
fully_connceted_1 (Dense)    (None, 1024)              2098176   
_________________________________________________________________
dropout_5 (Dropout)          (None, 1024)              0         
_________________________________________________________________
fully_connected_2 (Dense)    (None, 1024)              1049600   
_________________________________________________________________
fully_connected_3 (Dense)    (None, 85)                87125     
Total params: 3,234,901
Trainable params: 3,234,901
Non-trainable params: 0
_________________________________________________________________


In [46]:
build_discriminator()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dis_input (InputLayer)       (None, 2048)              0         
_________________________________________________________________
fully_connceted_1_dis (Dense (None, 10)                20490     
_________________________________________________________________
fully_connceted_2_dis (Dense (None, 5)                 55        
_________________________________________________________________
final_dis (Dense)            (None, 1)                 6         
Total params: 20,551
Trainable params: 20,551
Non-trainable params: 0
_________________________________________________________________
